In [1]:
from parse_deck import parse_events_from_official
from parse_deck import reassign_category

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import re
from tqdm import tqdm

import pandas as pd
from collections import OrderedDict

import json
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [3]:
RUN_PARSE = False

In [4]:
# decks = {
#     deck category 1: [
#         {
#             deck_link: "",
#             deck_code: "",
#             pokemons: {},
#             tools: {},
#             supporters: {},
#             stages: {},
#             energies: {},
#             rank: 1,
#             num_people: 32,
#             date: datetime
#         }, ...
#     ],
#     deck category 2:...
# }
all_categories = {
    "simple": ["ルギアVSTAR", "ミュウVMAX", "ムゲンダイナVMAX", "キュレムVMAX"],  # if this card exists, then we could decide the category
    "complicate": ["ゾロア", "レジドラゴVSTAR", "ギラティナVSTAR"]
}

decks = {}
store_file_name = 'store.json'
if os.path.exists(store_file_name):
    with open(store_file_name, 'r') as f:
        decks = json.load(f)
        
decks = reassign_category(decks, all_categories)
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])

dict_keys(['others', 'キュレムVMAX', 'レジ', 'LTB ヤミラミ リザードン', 'LTB リザードン', 'LTB ウッウ', 'LTB', 'ムゲンダイナVMAX', 'Other Lost', 'ルギアVSTAR', 'ミュウVMAX', 'LOST ギラティナVSTAR', 'レジドラゴVSTAR', 'アルレジドラゴVSTAR', 'ゾロア', 'Other ギラティナVSTAR'])


In [5]:
len(store_code_list)

1935

In [6]:
# parse events and save to json
if RUN_PARSE:
    parse_events_from_official(decks, all_categories, store_code_list) #, page_limit=30, event_limit=1000)

with open(store_file_name, 'w') as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [7]:
# modified categories
decks.keys()

dict_keys(['others', 'キュレムVMAX', 'レジ', 'LTB ヤミラミ リザードン', 'LTB リザードン', 'LTB ウッウ', 'LTB', 'ムゲンダイナVMAX', 'Other Lost', 'ルギアVSTAR', 'ミュウVMAX', 'LOST ギラティナVSTAR', 'レジドラゴVSTAR', 'アルレジドラゴVSTAR', 'ゾロア', 'Other ギラティナVSTAR'])

In [24]:
target_category = "LTB"
df_list = []
common_cols = ["date", "num_people", "rank"]

start_date = "2022年11月12日"  # include
end_date = "2022年12月09日"  # include

for card_type in ["pokemons", "tools", "supporters", "stages", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[target_category]):
        deck_link = deck["deck_link"]  # row id
        if deck["date"] < start_date or deck["date"] > end_date:
            continue
        pokecard = OrderedDict()
        pokecard["date"] = deck["date"]
        pokecard["num_people"] = deck["num_people"]
        pokecard["rank"] = deck["rank"]
        pokecard.update(deck[card_type])
        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_link])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_link])])
    df = df.fillna(0)
    
    # sort rows by date
    df = df.sort_values(by=['date'], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols: col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis='rows', numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis='rows')
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks
    
    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows-1, num_rows-2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]
    
    # sort cols by pick rate
    df = df.sort_values('pick_rate', axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols: col_list.remove(c)
    df = df[common_cols + col_list]
    
    # store
    df_list.append(df)

In [25]:
len(df_list[0]) - 2

79

In [26]:
df_list[0].to_csv(f"{target_category}-pokecard-{start_date}-{end_date}.csv")
df_list[0]

,date,num_people,rank,キュワワー,ウッウ,かがやくゲッコウガ,マナフィ,ヤミラミ,ザマゼンタ,ガラル ジグザグマ,ライコウ,カビゴン,ドラピオンV,ネオラントV,カイオーガ,コオリッポ,ザシアン,ヤレユータン,ライコウV,サンダー,ジガルデ,レックウザ,ヒスイ ヌメルゴンV,レシラム,ザシアンV,オリジンディアルガVSTAR,ヒスイ ヌメルゴンVSTAR,オリジンディアルガV,ガラル サンダーV,ゼクロム,ジュラルドンV,フリーザー,ゼラオラ,ミルタンク,ガラル ファイヤー,テラキオン,ジュペッタ,カゲボウズ,クロバットV,ザシアンVSTAR,ジャローダV,ジュラルドンVMAX,ジャローダVSTAR,レジエレキ,ミミッキュ,レントラー
pick_rate,NaN,NaN,NaN,1.0,1.000000,1.0,0.924051,0.911392,0.493671,0.468354,0.417722,0.367089,0.367089,0.341772,0.291139,0.189873,0.189873,0.177215,0.151899,0.075949,0.075949,0.050633,0.037975,0.037975,0.037975,0.037975,0.037975,0.037975,0.037975,0.025316,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658,0.012658
avg_num_used,NaN,NaN,NaN,4.0,1.772152,1.0,1.027397,1.944444,1.333333,1.000000,1.727273,1.034483,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.166667,1.000000,1.250000,3.000000,2.000000,1.333333,2.000000,3.000000,2.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,1.000000,2.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/k55FF1-ABgcmZ-VdFkFk,2022年12月08日(木),126.0,2.0,4.0,2.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kFkVdf-CejxBy-VFVfdk,2022年12月07日(水),32.0,2.0,4.0,2.000000,1.0,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/nggiLP-v3Hkwy-nninHg,2022年12月07日(水),72.0,5.0,4.0,2.000000,1.0,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/5VdFkV-m0JvCL-vF1kkk,2022年12月07日(水),72.0,3.0,4.0,2.000000,1.0,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/K88YJ4-L19Ku6-88x88c,2022年12月07日(水),32.0,3.0,4.0,1.000000,1.0,1.000000,2.000000,2.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/wFVbkV-OUFJpk-FFkdfk,2022年12月06日(火),126.0,5.0,4.0,1.000000,1.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000
https://

In [27]:
df_list[1].to_csv(f"{target_category}-tool-{start_date}-{end_date}.csv")
df_list[1]

,date,num_people,rank,あなぬけのヒモ,回収ネット,ミラージュゲート,ふつうのつりざお,バトルVIPパス,クイックボール,ふうせん,エネルギーリサイクル,ヒスイのヘビーボール,こだわりベルト,いれかえカート,やまびこホーン,ポケギア3.0,ロストスイーパー,ポケモンいれかえ,空の封印石,クロススイッチャー,ハイパーボール,霧の水晶,ビッグパラソル,大地の封印石,メタルソーサー,キャンセルコロン,望遠スコープ,森の封印石,スマホロトム,レベルボール
pick_rate,NaN,NaN,NaN,1.000000,1.0,1.000000,0.987342,0.962025,0.962025,0.822785,0.772152,0.64557,0.341772,0.329114,0.329114,0.164557,0.139241,0.088608,0.075949,0.075949,0.063291,0.037975,0.037975,0.037975,0.037975,0.037975,0.025316,0.025316,0.012658,0.012658
avg_num_used,NaN,NaN,NaN,2.708861,4.0,3.936709,1.910256,3.828947,2.947368,1.092308,1.475410,1.00000,1.259259,1.846154,1.076923,1.538462,1.090909,2.142857,1.000000,4.000000,1.600000,1.333333,1.666667,1.000000,4.000000,1.000000,1.000000,1.500000,2.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/k55FF1-ABgcmZ-VdFkFk,2022年12月08日(木),126.0,2.0,3.000000,4.0,4.000000,2.000000,4.000000,3.000000,1.000000,1.000000,1.00000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kFkVdf-CejxBy-VFVfdk,2022年12月07日(水),32.0,2.0,2.000000,4.0,4.000000,2.000000,0.000000,3.000000,1.000000,0.000000,1.00000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/nggiLP-v3Hkwy-nninHg,2022年12月07日(水),72.0,5.0,2.000000,4.0,4.000000,3.000000,4.000000,2.000000,1.000000,0.000000,1.00000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/5VdFkV-m0JvCL-vF1kkk,2022年12月07日(水),72.0,3.0,3.000000,4.0,4.000000,2.000000,4.000000,3.000000,2.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/K88YJ4-L19Ku6-88x88c,2022年12月07日(水),32.0,3.0,3.000000,4.0,4.000000,2.000000,4.000000,4.000000,1.000000,1.000000,1.00000,0.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/wFVbkV-OUFJpk-FFkdfk,2022年12月06日(火),126.0,5.0,1.000000,4.0,4.000000,1.000000,4.000000,2.000000,0.000000,1.000000,0.00000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ca8xx8-jlMYuG-c88xGJ,2022年12月06日(火),126.0,5.0,2.000000,4.0,4.000000,2.000000,4.000000,3.000000,1.000000,2.000000,1.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/9ngngL-aBuupf-nNnHnn,2022年12月04日(日),32.0,5.0,3.000000,4.0,4.000000,2.000000,4.000000,4.000000,1.000000,1.000000,1.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [28]:
df_list[2].to_csv(f"{target_category}-supporter-{start_date}-{end_date}.csv")
df_list[2]

,date,num_people,rank,アクロマの実験,ボスの指令,キバナ,ツツジ,とりつかい,クララ,セレナ,マリィ,ネジキ,カイ
pick_rate,NaN,NaN,NaN,1.0,0.784810,0.531646,0.291139,0.253165,0.113924,0.075949,0.050633,0.025316,0.012658
avg_num_used,NaN,NaN,NaN,4.0,1.451613,1.428571,1.086957,1.650000,1.000000,1.500000,1.000000,1.000000,2.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/k55FF1-ABgcmZ-VdFkFk,2022年12月08日(木),126.0,2.0,4.0,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kFkVdf-CejxBy-VFVfdk,2022年12月07日(水),32.0,2.0,4.0,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/nggiLP-v3Hkwy-nninHg,2022年12月07日(水),72.0,5.0,4.0,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/5VdFkV-m0JvCL-vF1kkk,2022年12月07日(水),72.0,3.0,4.0,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/K88YJ4-L19Ku6-88x88c,2022年12月07日(水),32.0,3.0,4.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/wFVbkV-OUFJpk-FFkdfk,2022年12月06日(火),126.0,5.0,4.0,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ca8xx8-jlMYuG-c88xGJ,2022年12月06日(火),126.0,5.0,4.0,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/9ngngL-aBuupf-nNnHnn,2022年12月04日(日),32.0,5.0,4.0,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [29]:
df_list[3].to_csv(f"{target_category}-stage-{start_date}-{end_date}.csv")
df_list[3]

,date,num_people,rank,トレーニングコート,シンオウ神殿,ポケストップ,崩れたスタジアム,頂への雪道,ロストシティ,エイチ湖
pick_rate,NaN,NaN,NaN,0.556962,0.392405,0.050633,0.050633,0.037975,0.025316,0.012658
avg_num_used,NaN,NaN,NaN,1.318182,1.258065,1.000000,1.000000,1.333333,1.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/k55FF1-ABgcmZ-VdFkFk,2022年12月08日(木),126.0,2.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kFkVdf-CejxBy-VFVfdk,2022年12月07日(水),32.0,2.0,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/nggiLP-v3Hkwy-nninHg,2022年12月07日(水),72.0,5.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/5VdFkV-m0JvCL-vF1kkk,2022年12月07日(水),72.0,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/K88YJ4-L19Ku6-88x88c,2022年12月07日(水),32.0,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/wFVbkV-OUFJpk-FFkdfk,2022年12月06日(火),126.0,5.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ca8xx8-jlMYuG-c88xGJ,2022年12月06日(火),126.0,5.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/9ngngL-aBuupf-nNnHnn,2022年12月04日(日),32.0,5.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [30]:
df_list[4].to_csv(f"{target_category}-energy-{start_date}-{end_date}.csv")
df_list[4]

,date,num_people,rank,基本超エネルギー,基本雷エネルギー,基本鋼エネルギー,基本水エネルギー,キャプチャーエネルギー,基本草エネルギー,基本闘エネルギー,基本炎エネルギー,基本悪エネルギー,Vガードエネルギー,スピード雷エネルギー,コーティング鋼エネルギー,ウォッシュ水エネルギー
pick_rate,NaN,NaN,NaN,0.911392,0.683544,0.620253,0.607595,0.518987,0.455696,0.177215,0.037975,0.037975,0.025316,0.012658,0.012658,0.012658
avg_num_used,NaN,NaN,NaN,2.847222,2.629630,3.897959,4.208333,2.292683,2.666667,2.000000,2.333333,2.666667,2.000000,2.000000,2.000000,1.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/k55FF1-ABgcmZ-VdFkFk,2022年12月08日(木),126.0,2.0,3.000000,2.000000,3.000000,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/kFkVdf-CejxBy-VFVfdk,2022年12月07日(水),32.0,2.0,2.000000,2.000000,4.000000,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/nggiLP-v3Hkwy-nninHg,2022年12月07日(水),72.0,5.0,2.000000,2.000000,4.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/5VdFkV-m0JvCL-vF1kkk,2022年12月07日(水),72.0,3.0,2.000000,3.000000,3.000000,0.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/K88YJ4-L19Ku6-88x88c,2022年12月07日(水),32.0,3.0,3.000000,2.000000,4.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/wFVbkV-OUFJpk-FFkdfk,2022年12月06日(火),126.0,5.0,2.000000,0.000000,3.000000,3.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/ca8xx8-jlMYuG-c88xGJ,2022年12月06日(火),126.0,5.0,2.000000,2.000000,0.000000,4.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
https://www.pokemon-card.com/deck/confirm.html/deckID/9ngngL-aBuupf-nNnHnn,2022年12月04日(日),32.0,5.0,1.000000,3.000000,3.000000,0.000000,0.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
len(decks["others"])

422